# Monkey Species Identifier

**Initialization**

In [1]:
import numpy as np
import joblib

import matplotlib.pyplot as plt

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.applications.xception import Xception
from google.colab import drive

from sklearn.metrics import precision_recall_fscore_support

Using TensorFlow backend.


In [2]:
# Mount google drive
drive.mount("drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at drive


**Load Dataset**

In [0]:
DATASET_PATH = "drive/My Drive/dataset/dataset_monkey.joblib"
X_train, Y_train = joblib.load(DATASET_PATH)

DATASET_PATH = "drive/My Drive/dataset/dataset_monkey_test.joblib"
X_test, Y_test = joblib.load(DATASET_PATH)

In [4]:
X_test.shape[0]

816

**Set variables**

In [0]:
HEIGHT = X_train.shape[1]
WIDTH = X_train.shape[2]
CHANNELS = X_train.shape[3]

TRAIN_SAMPLES = X_train.shape[0]
TEST_SAMPLES = X_test.shape[0]
BATCH_SIZE = 32
CLASSES = 10
EPOCH_STEPS = int(TRAIN_SAMPLES / BATCH_SIZE)
EPOCHS = 50

NN_OUTPUT_HEIGHT = 7
NN_OUTPUT_WIDTH = 7
NN_OUTPUT_CHANNELS = 2048

In [0]:
nn_model = Xception(
  weights="imagenet",
  input_shape=(HEIGHT, WIDTH, CHANNELS),
  include_top=False
)

**Extract Features for training dataset**

In [0]:
X_train_reduced = np.zeros((TRAIN_SAMPLES, NN_OUTPUT_HEIGHT, NN_OUTPUT_WIDTH, NN_OUTPUT_CHANNELS), dtype=np.float32)

for i in range(TRAIN_SAMPLES):
  X_train_reduced[i, :, :, :] = nn_model.predict(X_train[i:i+1, :, :, :])

# Cleanup memory
del X_train
  
X_train = np.reshape(X_train_reduced, (TRAIN_SAMPLES, 7 * 7 * 2048))

**Extract features for test dataset**

In [0]:
X_test_reduced = np.zeros((TEST_SAMPLES, NN_OUTPUT_HEIGHT, NN_OUTPUT_WIDTH, NN_OUTPUT_CHANNELS), dtype=np.float32)

for i in range(TEST_SAMPLES):
  X_test_reduced[i, :, :, :] = nn_model.predict(X_test[i:i+1, :, :, :])

# Cleanup memory
del X_test
  
X_test = np.reshape(X_test_reduced, (TEST_SAMPLES, 7 * 7 * 2048))

**Construct Neural Network model**

In [0]:
model = Sequential()
model.add(Dense(512, activation='relu', input_dim=7*7*2048))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['acc'])

**Train the model**

In [12]:
history = model.fit(
  X_train, 
  Y_train, 
  epochs=25,
  batch_size=BATCH_SIZE,
  shuffle=True,
  validation_split=0.1
)

Train on 2961 samples, validate on 330 samples
Epoch 1/25
2961/2961 [==============================] - 9s 3ms/step - loss: 4.6755 - acc: 0.6842 - val_loss: 2.9895 - val_acc: 0.7970
Epoch 2/25
2961/2961 [==============================] - 7s 2ms/step - loss: 2.9430 - acc: 0.8099 - val_loss: 2.3923 - val_acc: 0.8485
Epoch 3/25
2961/2961 [==============================] - 7s 2ms/step - loss: 2.3064 - acc: 0.8494 - val_loss: 1.4046 - val_acc: 0.9030
Epoch 4/25
2961/2961 [==============================] - 7s 2ms/step - loss: 1.0440 - acc: 0.9314 - val_loss: 0.7336 - val_acc: 0.9515
Epoch 5/25
2961/2961 [==============================] - 7s 2ms/step - loss: 1.1029 - acc: 0.9267 - val_loss: 1.0730 - val_acc: 0.9303
Epoch 6/25
2961/2961 [==============================] - 7s 2ms/step - loss: 1.2492 - acc: 0.9176 - val_loss: 1.4942 - val_acc: 0.9000
Epoch 7/25
2961/2961 [==============================] - 7s 2ms/step - loss: 0.9780 - acc: 0.9362 - val_loss: 0.7586 - val_acc: 0.9485
Epoch 8/25
2961

In [0]:
# Predict values
Y_pred = model.predict(X_test)

In [0]:
Y_pred_labels = [x.argmax() for x in Y_pred]
Y_test_labels = [x.argmax() for x in Y_test]

In [18]:
print(precision_recall_fscore_support(Y_test_labels, Y_pred_labels))

(array([0.98717949, 0.97530864, 1.        , 0.97333333, 0.9625    ,
       1.        , 0.47852761, 0.70338983, 1.        , 1.        ]), array([0.98717949, 0.94047619, 0.43209877, 0.81111111, 0.98717949,
       0.9047619 , 1.        , 0.98809524, 0.43209877, 0.96153846]), array([0.98717949, 0.95757576, 0.60344828, 0.88484848, 0.97468354,
       0.95      , 0.6473029 , 0.82178218, 0.60344828, 0.98039216]), array([78, 84, 81, 90, 78, 84, 78, 84, 81, 78]))
